In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import plot_model

df = pd.read_csv('3_year_recidivism_elaborated_2.csv')
df

,Fiscal Year Released,Recidivism Reporting Year,Race - Ethnicity,Age At Release,Convicting Offense Classification,Convicting Offense Type,Main Supervising District,Release type: Paroled to Detainer united,Part of Target Population,Recidivism - Return to Prison numeric
0,2010,2013,White - Non-Hispanic,Under 25,D Felony,Violent,4JD,Parole,Yes,1
1,2010,2013,White - Non-Hispanic,55 and Older,D Felony,Public Order,7JD,Parole,Yes,1
2,2010,2013,White - Non-Hispanic,25-34,D Felony,Property,5JD,Parole,Yes,1
3,2010,2013,White - Non-Hispanic,55 and Older,C Felony,Drug,8JD,Parole,Yes,1
4,2010,2013,Black - Non-Hispanic,25-34,D Felony,Drug,3JD,Parole,Yes,1
...,...,...,...,...,...,...,...,...,...,...
26015,2015,2018,White - Hispanic,25-34,C Felony,Violent,Not Mentioned,Discharged End of Sentence,Yes,0
26016,2015,2018,White - Non-Hispanic,25-34,D Felony,Property,5JD,Paroled to Detainer,No,0
26017,2015,2018,Black - Non-Hispanic,Under 25,Aggravated Misdemeanor,Violent,1JD,Discharged End of Sentence,Yes,0
26018,2015,2018,White - Non-Hispanic,Under 25,D Felony,Drug,5JD,Parole,No,0


In [2]:
df.describe()

,Fiscal Year Released,Recidivism Reporting Year,Recidivism - Return to Prison numeric
count,26020.000000,26020.000000,26020.000000
mean,2012.600769,2015.600769,0.333628
std,1.661028,1.661028,0.471518
min,2010.000000,2013.000000,0.000000
25%,2011.000000,2014.000000,0.000000
50%,2013.000000,2016.000000,0.000000
75%,2014.000000,2017.000000,1.000000
max,2015.000000,2018.000000,1.000000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26020 entries, 0 to 26019
Data columns (total 10 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   Fiscal Year Released                      26020 non-null  int64 
 1   Recidivism Reporting Year                 26020 non-null  int64 
 2   Race - Ethnicity                          26020 non-null  object
 3   Age At Release                            26020 non-null  object
 4   Convicting Offense Classification         26020 non-null  object
 5   Convicting Offense Type                   26020 non-null  object
 6   Main Supervising District                 26020 non-null  object
 7   Release type: Paroled to Detainer united  26020 non-null  object
 8   Part of Target Population                 26020 non-null  object
 9   Recidivism - Return to Prison numeric     26020 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.0+ MB


In [4]:
from sklearn.preprocessing import OneHotEncoder

x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

enc = OneHotEncoder(sparse=False)
one_hot_x = enc.fit_transform(x)

In [5]:
enc.categories_

[array([2010, 2011, 2012, 2013, 2014, 2015], dtype=object),
 array([2013, 2014, 2015, 2016, 2017, 2018], dtype=object),
 array(['American Indian or Alaska Native - Hispanic',
        'American Indian or Alaska Native - Non-Hispanic',
        'Asian or Pacific Islander - Hispanic',
        'Asian or Pacific Islander - Non-Hispanic', 'Black -',
        'Black - Hispanic', 'Black - Non-Hispanic', 'N/A -',
        'Not Mentioned', 'White -', 'White - Hispanic',
        'White - Non-Hispanic'], dtype=object),
 array(['25-34', '35-44', '45-54', '55 and Older', 'Not Mentioned',
        'Under 25'], dtype=object),
 array(['A Felony', 'Aggravated Misdemeanor', 'B Felony', 'C Felony',
        'D Felony', 'Felony - Enhanced',
        'Felony - Enhancement to Original Penalty',
        'Felony - Mandatory Minimum', 'Other Felony',
        'Other Felony (Old Code)', 'Other Misdemeanor',
        'Serious Misdemeanor', 'Sexual Predator Community Supervision',
        'Simple Misdemeanor', 'Special Se

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4)

for train_index, combo_index in sss.split(one_hot_x, y):
    train_x, combo_x = one_hot_x[train_index], one_hot_x[combo_index]
    train_y, combo_y = y[train_index], y[combo_index]

In [7]:
combo_x.shape

(10408, 68)

In [8]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5)

for test_index, valid_index in sss.split(combo_x, combo_y):
    test_x, valid_x = combo_x[test_index], combo_x[valid_index]
    test_y, valid_y = combo_y[test_index], combo_y[valid_index]

In [9]:
len(valid_y)

5204

In [10]:
import torch

train_x, test_x, valid_x = map(torch.tensor, (train_x, test_x, valid_x))
train_y, test_y, valid_y = map(torch.tensor, (train_y, test_y, valid_y))

In [11]:
train_x = train_x.float()
test_x = test_x.float()
valid_x = valid_x.float()

In [12]:
train_x

tensor([[0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 1., 0.,  ..., 1., 0., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [13]:
train_x = tf.dtypes.cast(train_x, tf.float64)
test_x = tf.dtypes.cast(test_x, tf.float64)
valid_x = tf.dtypes.cast(valid_x, tf.float64)

train_y = tf.dtypes.cast(train_y, tf.int64)
test_y = tf.dtypes.cast(test_y, tf.int64)
valid_y = tf.dtypes.cast(valid_y, tf.int64)

# DNN

In [14]:
from torch.utils.data import Dataset

class dds(Dataset):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

In [15]:
from torch.utils.data import DataLoader

mydata = dds(train_x, train_y)
train_loader = DataLoader(dataset = mydata, batch_size=16)

In [16]:
train_loader

In [17]:
%load_ext tensorboard

In [18]:
from tensorboard.plugins.hparams import api as hp

In [19]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([40, 80, 120]))
HP_NUM_LAYERS = hp.HParam('num_layers', hp.Discrete([3, 5, 7, 9]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.3))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'rmsprop']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_NUM_LAYERS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [20]:
def train_test_model(hparams):
    
  dense_layers = [] #[Dense(16, activation='relu') for _ in range(hparams[HP_NUM_LAYERS])]

  for i in range(hparams[HP_NUM_LAYERS]):
        dense_layers.append(tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu))
    
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    dense_layers,
#     tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax),
  ])

  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )
    
  model.fit(train_x, train_y, epochs=1) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(test_x, test_y)
    
  plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

  return accuracy

In [21]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [22]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
        for num_layers in HP_NUM_LAYERS.domain.values:
          hparams = {
              HP_NUM_UNITS: num_units,
              HP_DROPOUT: dropout_rate,
              HP_OPTIMIZER: optimizer,
              HP_NUM_LAYERS: num_layers,
          }
          run_name = "run-%d" % session_num
          print('--- Starting trial: %s' % run_name)
          print({h.name: hparams[h] for h in hparams})
          run('logs/hparam_tuning/' + run_name, hparams)
          session_num += 1

--- Starting trial: run-0
{'num_units': 40, 'dropout': 0.1, 'optimizer': 'adam', 'num_layers': 3}


TypeError: The added layer must be an instance of class Layer. Received: layer=[<keras.layers.core.dense.Dense object at 0x000001B664A4CAC0>, <keras.layers.core.dense.Dense object at 0x000001B664A77D30>, <keras.layers.core.dense.Dense object at 0x000001B664AF41C0>] of type <class 'list'>.

In [ ]:
%tensorboard --logdir logs/hparam_tuning --host localhost